<h1> Task 1 </h1>
<h3> 2. Load the three JSON files and encode them as pandas dataframes. </h3>

In [53]:
import pandas as pd
import numpy as np

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
training_url = '/content/drive/MyDrive/Assignment 1/data/training.json'
validation_url = '/content/drive/MyDrive/Assignment 1/data/validation.json'
test_url = '/content/drive/MyDrive/Assignment 1/data/test.json'

In [56]:
training_set = pd.read_json(training_url, orient='index')
validation_set = pd.read_json(validation_url, orient='index')
test_set = pd.read_json(test_url, orient='index')

<h3> 3. Generate hard labels for Task 1 using majority voting and store them in a new dataframe column called `hard_label_task1`. <br>Items without a clear majority will be removed from the dataset. </h3>

In [57]:
def majority(l):
    y_count = l.count('YES')
    n_count = l.count('NO')

    if y_count == n_count:
        return pd.NaT

    if y_count > 3:
        return 'YES'

    return 'NO'

In [58]:
training_set['hard_label_task1'] = training_set['labels_task1'].apply(majority)
training_set.dropna(axis=0, inplace=True)

validation_set['hard_label_task1'] = validation_set['labels_task1'].apply(majority)
validation_set.dropna(axis=0, inplace=True)

test_set['hard_label_task1'] = test_set['labels_task1'].apply(majority)
test_set.dropna(axis=0, inplace=True)

<h3> 4. Filter the DataFrame to keep only rows where the `lang` column is `'en'`. </h3>

In [59]:
lang = 'en'

In [60]:
training_set = training_set[training_set['lang'] == lang]
validation_set = validation_set[validation_set['lang'] == lang]
test_set = test_set[test_set['lang'] == lang]

<h3> 5. Remove unwanted columns: Keep only `id_EXIST`, `lang`, `tweet`, and `hard_label_task1`. </h3>

In [61]:
training_set = training_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
validation_set = validation_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
test_set = test_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]

<h3> 6. Encode the `hard_label_task1` column: Use 1 to represent "YES" and 0 to represent "NO".</h3>

In [62]:
training_set['hard_label_task1'] = training_set['hard_label_task1'].map({'YES':1, 'NO':0})
validation_set['hard_label_task1'] = validation_set['hard_label_task1'].map({'YES':1, 'NO':0})
test_set['hard_label_task1'] = test_set['hard_label_task1'].map({'YES':1, 'NO':0})

In [63]:
training_set

,id_EXIST,lang,tweet,hard_label_task1
200002,200002,en,Writing a uni essay in my local pub with a cof...,1
200003,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,1
200006,200006,en,According to a customer I have plenty of time ...,1
200007,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",1
200008,200008,en,New to the shelves this week - looking forward...,0
...,...,...,...,...
203256,203256,en,idk why y’all bitches think having half your a...,1
203257,203257,en,This has been a part of an experiment with @Wo...,1
203258,203258,en,"""Take me already"" ""Not yet. You gotta be ready...",1
203259,203259,en,@clintneedcoffee why do you look like a whore?...,1


<h1> Task 2 </h1>

- **Remove emojis** from the tweets.
- **Remove hashtags** (e.g., `#example`).
- **Remove mentions** such as `@user`.
- **Remove URLs** from the tweets.
- **Remove special characters and symbols**.
- **Remove specific quote characters** (e.g., curly quotes).
- **Perform lemmatization** to reduce words to their base form.

In [64]:
!pip install emoji

import re
import emoji
from nltk.corpus import stopwords
import nltk

In [65]:
emojis_list = map(lambda x: ''.join(x.split()), emoji.EMOJI_DATA.keys())

EMOJI_RE = re.compile('|'.join(re.escape(p) for p in emojis_list))
HASHTAGS_RE = re.compile('#\w+')
MENTIONS_RE = re.compile('@\w+')
URL_RE = re.compile('(https|http)?:\/\/\S+')
#Qui ho aggiunto il punto sostituito dallo spazio perchè mi sembra che la maggior parte
#dei tweets ne traggono beneficio, poi TODO va provato raga
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;‘’“”\"\.]')
SPECIAL_CHARACTERS_RE = re.compile('&amp;')
GOOD_SYMBOLS_RE = re.compile('[^\w+ +]')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

In [66]:
def lower(text: str) -> str:
    return text.lower()

def remove_emojis(text: str) -> str:
    return EMOJI_RE.sub(' ',text)

def remove_hashtags(text: str) -> str:
    return HASHTAGS_RE.sub(' ', text)

def remove_mentions(text: str) -> str:
    return MENTIONS_RE.sub(' ', text)

def remove_url(text: str) -> str:
    return URL_RE.sub(' ',text)

def remove_special_characters(text: str) -> str:
    return SPECIAL_CHARACTERS_RE.sub('', text)

def replace_special_characters(text: str) -> str:
    return REPLACE_BY_SPACE_RE.sub(' ', text)

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the good symbols list (check regular expression)
    """
    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    """
    return text.strip()

We can observe that some hashtags in the form "#somethinghttps://" also removes the initial part of the link.

@rufinelix's account

In [67]:
from typing import List, Callable, Dict
from functools import reduce

PREPROCESSING_PIPELINE = [
                          lower,
                          remove_emojis,
                          remove_hashtags,
                          remove_mentions,
                          remove_url,
                          remove_special_characters,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text
                          ]
#Lui elimina anche le stopwords, poi TODO va provato raga

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

In [68]:
print('Pre-processing text...')

print()
print(f'[Debug] Before:\n{training_set.tweet.values[2]}')
print()

# Replace each sentence with its pre-processed version
training_set['tweet'] = training_set['tweet'].apply(lambda txt: text_prepare(txt))
validation_set['tweet'] = validation_set['tweet'].apply(lambda txt: text_prepare(txt))
test_set['tweet'] = test_set['tweet'].apply(lambda txt: text_prepare(txt))

print(f'[Debug] After:\n{training_set.tweet.values[2]}')
print()

print("Pre-processing completed!")

Pre-processing text...

[Debug] Before:
According to a customer I have plenty of time to go spent the Stirling coins he wants to pay me with, in Derry. "Just like any other woman, I'm sure of it." #EveryDaySexism in retail.

[Debug] After:
according customer plenty time go spent stirling coins wants pay derry like woman im sure retail

Pre-processing completed!


In [69]:
#Focus Focus Focus qui le contrazioni tipo I'm le accorpa, ma sotto la lemmatization con wordNet le
#scoppia. Se usassimo le stopwords le eliminerebbe direttamente e il problema non si porrebbe, quindi indovinate
#TODO va provato raga
training_set.iloc[:10]['tweet']

,tweet
200002,writing uni essay local pub coffee random old ...
200003,2021 1921 dont appreciate two rides team membe...
200006,according customer plenty time go spent stirli...
200007,blokes drink beer sorry arent bloke drink wine...
200008,new shelves week looking forward reading books
200010,guess fairly normal neanderthal
200011,means women usually end lower paid support wor...
200013,hi orla interesting piece 2 policy response be...
200015,dear god colette capable identifying sexism li...
200016,women home cooking family


In [70]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import WhitespaceTokenizer

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()

def get_wordnet_key(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def lem_text(text: str):
    tokens = tokenizer.tokenize(text)
    tagged = pos_tag(tokens)
    words = [lemmatizer.lemmatize(word, get_wordnet_key(tag)) for word, tag in tagged]
    return " ".join(words)

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
training_set['tweet'][:10]

,tweet
200002,writing uni essay local pub coffee random old ...
200003,2021 1921 dont appreciate two rides team membe...
200006,according customer plenty time go spent stirli...
200007,blokes drink beer sorry arent bloke drink wine...
200008,new shelves week looking forward reading books
200010,guess fairly normal neanderthal
200011,means women usually end lower paid support wor...
200013,hi orla interesting piece 2 policy response be...
200015,dear god colette capable identifying sexism li...
200016,women home cooking family


In [72]:
lem_train_texts = [lem_text(text) for text in training_set['tweet']]
lem_validation_texts = [lem_text(text) for text in validation_set['tweet']]
lem_test_texts = [lem_text(text) for text in test_set['tweet']]

In [73]:
lem_train_texts[:10]

['write uni essay local pub coffee random old man keep ask drunk question im try concentrate end good luck youll end get marry use anyway alive well',
 '2021 1921 dont appreciate two ride team member look behind ask man behind many party impress',
 'accord customer plenty time go spent stirling coin want pay derry like woman im sure retail',
 'bloke drink beer sorry arent bloke drink wine apparently alive well',
 'new shelf week look forward read book',
 'guess fairly normal neanderthal',
 'mean woman usually end low pay support work start change traditionalist notice unfairness previously hasnt bother',
 'hi orla interest piece 2 policy response believe earlyinlife prosecution punishment way go boy young men offend girl woman see zero tolerance approach institute',
 'dear god colette capable identify sexism literally anywhere good see develop female grandpa simpson',
 'woman home cooking family']

In [74]:
training_set['tweet'] = lem_train_texts
validation_set['tweet'] = lem_validation_texts
test_set['tweet'] = lem_test_texts

<h1> Task 3 </h1>

Embed words using **GloVe embeddings**. <br>
You are **free** to pick any embedding dimension.

In [75]:
from collections import OrderedDict

In [76]:
def build_vocabulary(df: pd.DataFrame) -> (Dict[int, str], Dict[str, int], List[str]):
    """
    Given a dataset, builds the corresponding word vocabulary.

    :param df: dataset from which we want to build the word vocabulary (pandas.DataFrame)
    :return:
      - word vocabulary: vocabulary index to word
      - inverse word vocabulary: word to vocabulary index
      - word listing: set of unique terms that build up the vocabulary
    """
    idx_to_word = OrderedDict()
    word_to_idx = OrderedDict()

    curr_idx = 0
    for sentence in df.tweet:
        tokens = sentence.split()
        for token in tokens:
            if token not in word_to_idx:
                word_to_idx[token] = curr_idx
                idx_to_word[curr_idx] = token
                curr_idx += 1

    word_to_idx['UNK'] = curr_idx
    idx_to_word[curr_idx] = 'UNK'

    word_listing = list(idx_to_word.values())
    return idx_to_word, word_to_idx, word_listing

In [77]:
idx_to_word, word_to_idx, word_listing = build_vocabulary(training_set)
print(f'[Debug] Index -> Word vocabulary size: {len(idx_to_word)}')
print(f'[Debug] Word -> Index vocabulary size: {len(word_to_idx)}')
print(f'[Debug] Some words: {[(idx_to_word[idx], idx) for idx in np.arange(10) + 1]}')

[Debug] Index -> Word vocabulary size: 9363
[Debug] Word -> Index vocabulary size: 9363
[Debug] Some words: [('uni', 1), ('essay', 2), ('local', 3), ('pub', 4), ('coffee', 5), ('random', 6), ('old', 7), ('man', 8), ('keep', 9), ('ask', 10)]


In [78]:
idx_to_word, word_to_idx, word_listing = build_vocabulary(training_set)

In [79]:
import gensim
import gensim.downloader as gloader

In [80]:
embedding_dimension = 50
download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
emb_model = gloader.load(download_path)

In [81]:
def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                    word_listing: List[str]):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    embedding_vocabulary = set(embedding_model.key_to_index.keys())
    oov = set(word_listing).difference(embedding_vocabulary)
    return list(oov)

In [82]:
oov_terms = check_OOV_terms(emb_model, word_listing)
oov_percentage = float(len(oov_terms)) * 100 / len(word_listing)
print(f"Total OOV terms: {len(oov_terms)} ({oov_percentage:.2f}%)")

Total OOV terms: 1340 (14.31%)


In [83]:
def build_embedding_matrix(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                           embedding_dimension: int,
                           word_to_idx: Dict[str, int],
                           vocab_size: int,
                           oov_terms: List[str]) -> np.ndarray:
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary
    :param oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)
    for word, idx in word_to_idx.items():
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [84]:
embedding_matrix = build_embedding_matrix(emb_model, embedding_dimension, word_to_idx, len(word_to_idx), oov_terms)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (9363, 50)


#### Creating an extra column for each dataset to store all the token ids

In [85]:
def preprocess_text(sentence: str, word_to_idx: Dict[str, int]) -> List[int]:
    """
    Maps tokens in a sentence to vocabulary indices, replacing OOV terms with 'UNK'.

    :param sentence: input sentence (str)
    :param word_to_idx: word-to-index mapping (dict)
    :return: list of token indices (list of int)
    """
    tokens = sentence.split()
    return [word_to_idx.get(token, word_to_idx['UNK']) for token in tokens]

training_set['processed'] = training_set['tweet'].apply(
    lambda x: preprocess_text(x, word_to_idx)
)

# Validation/Test: Preprocess to handle OOV terms
validation_set['processed'] = validation_set['tweet'].apply(
    lambda x: preprocess_text(x, word_to_idx)
)
test_set['processed'] = test_set['tweet'].apply(
    lambda x: preprocess_text(x, word_to_idx)
)

# Inspect a test sentence
print("Original:", test_set['tweet'].iloc[0])
print("Processed indices:", test_set['processed'].iloc[0])
print("Embedding for 'UNK':", embedding_matrix[word_to_idx['UNK']])

Original: 1st day pool beautiful sunday ny bad dare go due frigid water temp last 45 min figure prove point
Processed indices: [5076, 299, 9094, 1952, 1176, 2871, 432, 2310, 43, 668, 4916, 4763, 4990, 1055, 9362, 2277, 775, 479, 534]
Embedding for 'UNK': [ 0.01806087 -0.00517448 -0.03251405  0.02877776 -0.01102105  0.01444829
  0.03964875 -0.03559718  0.02460842 -0.04382962  0.01952896 -0.01317036
  0.04516917  0.0422252   0.02840588  0.02411183 -0.0449602   0.01511133
 -0.01043571 -0.0226937   0.01184831  0.03518415 -0.00442959  0.04760322
 -0.02000306 -0.0315448   0.02480833 -0.0203414  -0.02227696  0.04458721
  0.04528634 -0.01116554  0.00918736  0.01863066 -0.03175552 -0.00786053
 -0.0239286  -0.04398062 -0.03539643  0.01477463 -0.04517939 -0.01180867
 -0.04244076 -0.00156193  0.0477278   0.01793771 -0.03798958  0.01222676
  0.00223762 -0.01398266]


In [86]:
training_set['processed']

,processed
200002,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
200003,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 10, 8..."
200006,"[39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5..."
200007,"[54, 55, 56, 57, 58, 54, 55, 59, 60, 24, 25]"
200008,"[61, 62, 63, 34, 64, 65, 66]"
...,...
203256,"[902, 271, 523, 1084, 1708, 1827, 1919, 34, 50..."
203257,"[263, 3537, 13, 990, 1098, 2181, 1362, 6513, 3..."
203258,"[623, 361, 705, 188, 1151, 13, 6401, 370, 6380..."
203259,"[34, 50, 2238, 9360]"


In [87]:
embedding_matrix

array([[-0.065611  ,  0.18878   ,  0.38947   , ...,  0.50332   ,
        -0.37874   ,  0.78717   ],
       [-0.48424   ,  0.10052   , -0.20392   , ...,  0.528     ,
         0.47025   , -0.099937  ],
       [-0.73514   ,  1.0877    , -1.0185    , ...,  0.40231   ,
         0.07081   ,  0.24598   ],
       ...,
       [ 0.94112   ,  0.072009  ,  0.054059  , ..., -1.035     ,
        -0.1885    ,  0.35589   ],
       [ 0.17566   ,  0.40873   , -0.3003    , ..., -0.67123   ,
        -0.91092   ,  0.88043   ],
       [ 0.01806087, -0.00517448, -0.03251405, ...,  0.01222676,
         0.00223762, -0.01398266]], dtype=float32)

<h1> TASK 4 </h1>

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.

In [88]:
from keras.layers import LSTM, Bidirectional, Dense, Embedding
from keras.models import Sequential

#### Padding sentences to achieve the same length for each of them

In [96]:
from keras.preprocessing.sequence import pad_sequences

max_length = 100  # Define maximum sequence length
train_padded = pad_sequences(training_set['processed'], maxlen=max_length, padding='post')
validation_padded = pad_sequences(validation_set['processed'], maxlen=max_length, padding='post')
test_padded = pad_sequences(test_set['processed'], maxlen=max_length, padding='post')

### Baseline

In [101]:
vocab_size = len(word_to_idx)

embedding_layer = Embedding(
                            input_dim=vocab_size,
                            output_dim=embedding_dimension,
                            weights=[embedding_matrix],
                            mask_zero=True,                   # automatically masks padding tokens
                            name='encoder_embedding'
                            #trainable=False
                            )

# defining a bidirectional LSTM
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dense(1, activation='sigmoid'))

In [100]:
train_padded.shape

(2870, 100)

In [102]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #binary because we have a binary classification task
model.fit(train_padded, training_set['hard_label_task1'], epochs=10, batch_size=32, validation_data=(validation_padded, validation_set['hard_label_task1']))

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 166ms/step - accuracy: 0.6155 - loss: 0.6358 - val_accuracy: 0.7468 - val_loss: 0.5123
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - accuracy: 0.7617 - loss: 0.5035 - val_accuracy: 0.7595 - val_loss: 0.4720
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.8249 - loss: 0.4011 - val_accuracy: 0.7848 - val_loss: 0.4375
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 173ms/step - accuracy: 0.8690 - loss: 0.3258 - val_accuracy: 0.7911 - val_loss: 0.4737
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accuracy: 0.9065 - loss: 0.2418 - val_accuracy: 0.7722 - val_loss: 0.5491
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 163ms/step - accuracy: 0.9429 - loss: 0.1596 - val_accuracy: 0.8291 - val_loss: 0.4136
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 15s 163ms/step - accuracy: 0.9775 - loss: 0.0840 - val_accuracy: 0.8038 - val_loss: 0.4736
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - accuracy: 0.9770 - loss: 0.0695 - val_accu

### Model 1